In [ ]:
import astropy.constants as const
import astropy.units as u
import click
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.time import Time
from scipy.stats import lognorm
from tqdm import tqdm as Tqdm

import fastimgproto.visibility as visibility
from fastimgproto.skymodel.helpers import SkySource
from fastimgproto.telescope.readymade import Meerkat

In [ ]:
nstep = 10
output_npz = 'simdata_nstep10.npz'

pointing_centre = SkyCoord(0 * u.deg, -30 * u.deg)

telescope = Meerkat()
obs_central_frequency = 0.03 * u.GHz
wavelength = const.c / obs_central_frequency
transit_time = telescope.next_transit(pointing_centre.ra,
                                      start_time=Time('2017-01-01'))
obs_times = transit_time + np.linspace(-4, 4, nstep) * u.hr - 0 * u.hr
uvw_m = telescope.uvw_tracking_skycoord(pointing_centre, obs_times)

# From here on we use UVW as multiples of wavelength, lambda:
uvw_lambda = (uvw_m / wavelength).to(u.dimensionless_unscaled).value
vis_noise_level = 0.001 * u.Jy

# source_list = get_lsm(field_of_view)
# source_list = get_spiral_source_test_pattern(field_of_view)
extra_src_position = SkyCoord(ra=pointing_centre.ra + 0.01 * u.deg,
                              dec=pointing_centre.dec + 0.01 * u.deg, )

steady_sources = [
    SkySource(pointing_centre, flux=1 * u.Jy),
    SkySource(extra_src_position, flux=0.4 * u.Jy),
]

transient_posn = SkyCoord(
    ra=pointing_centre.ra - 2.0 * u.deg,
    dec=pointing_centre.dec - 0.0 * u.deg)
transient_posn2 = SkyCoord(
    ra=pointing_centre.ra - 4.0 * u.deg,
    dec=pointing_centre.dec - 0.0 * u.deg)
transient_posn3 = SkyCoord(
    ra=pointing_centre.ra - 6.0 * u.deg,
    dec=pointing_centre.dec - 0.0 * u.deg)
transient_posn4 = SkyCoord(
    ra=pointing_centre.ra - 8.0 * u.deg,
    dec=pointing_centre.dec - 0.0 * u.deg)
transient_posn5 = SkyCoord(
    ra=pointing_centre.ra - 10.0 * u.deg,
    dec=pointing_centre.dec - 0.0 * u.deg)

transient_sources = [
    SkySource(position=transient_posn, flux=1 * u.Jy),
    SkySource(position=transient_posn2, flux=1 * u.Jy),
    SkySource(position=transient_posn3, flux=1 * u.Jy),
    SkySource(position=transient_posn4, flux=1 * u.Jy),
    SkySource(position=transient_posn5, flux=1 * u.Jy),
]

#all_sources = steady_sources + transient_sources
all_sources = transient_sources

# Now use UVW to generate visibilities from scratch...
# Store l,m cosines & fluxes for skymodel (known sources only):
local_skymodel = []
for src in steady_sources:
    l, m = visibility.calculate_direction_cosines(pointing_centre, src)
    local_skymodel.append((l,m,src.flux.to(u.Jy).value))
local_skymodel = np.asarray(local_skymodel,dtype=np.float_)

# Simulate incoming data; includes transient sources, noise:
data_vis = visibility.visibilities_for_source_list(
    pointing_centre, all_sources, uvw_lambda)
data_vis = visibility.add_gaussian_noise(vis_noise_level, data_vis)

# Generate some SNR weights
# Current implementation has no theoretical basis, just picked an arbitrary
# plausible-looking distribution:
#snr_weights = np.minimum(1.0, lognorm.rvs(s=0.25, size=len(data_vis)))

#Alternatively, just assume perfect data:
snr_weights = np.ones_like(data_vis,dtype=np.float_)

In [ ]:
np.savez(output_npz,
         uvw_lambda=uvw_lambda,
         skymodel=local_skymodel,
         vis=data_vis,
         snr_weights=snr_weights,
         )